In [74]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

Training data

In [82]:
TRAIN_DATA = [
    ("USTHB is the biggest university in algeria.",{'entities': [(0, 5, 'ORG')]}),
    ("algiers has many universities on of them is USTHB",{'entities': [(43, 49, 'ORG')]}),
    ("LRIA is a research laboratory in the IT faculty at USTHB specialized in AI.",{'entities': [(0, 4, "ORG")]}),
    ("LRIA was founded in 2000",{'entities': [(0, 4, "ORG")]})
    ]

# Training a pos tagger

train a pos tagger to recognize a new entity “USTHB” and “LRIA” as ORG,  

In [84]:
#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")

        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 4/4 [00:00<00:00, 2039.78it/s]


In [87]:
import os
os.system('python -m spacy init fill-config tagger_config.cfg config.cfg')

0

In [88]:
import os
os.system('python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy')

1

In [93]:
nlp = spacy.load("./output/model-best")

doc = nlp("USTHB is the biggest university in algeria. and LRIA is an AI research laboratory at USTHB")

for ent in doc.ents:
    print(ent.text,"->", ent.label_)

USTHB -> ORG
LRIA -> ORG
USTHB -> ORG


# Training a ner Tagger

 ner tagger to recognize a new entity label called “COLOR”